In [ ]:

def __Price_Check(Target_Expcode) :

    print("# Price_Check Start : %s, hcode : %s" % (datetime.datetime.today().strftime("%H:%M:%S"), Target_Expcode))

    class XAQueryEventHandlerT1101:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT1101.query_state = 1
            
        def OnReceiveMessage(self, error, code, message):
            print("OnreceiveMessage", error, code, message)

    instXAQueryT1101 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1101)
    instXAQueryT1101.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t1101.res"
    instXAQueryT1101.SetFieldData("t1101InBlock", "shcode", 0, Target_Expcode)
    
    instXAQueryT1101.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT1101.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    hname = instXAQueryT1101.GetFieldData("t1101OutBlock", "hname", 0)
    price = instXAQueryT1101.GetFieldData("t1101OutBlock", "price", 0) # ---> 가격
    diff = instXAQueryT1101.GetFieldData("t1101OutBlock", "diff", 0) # ---> 등락율
    bidho2 = instXAQueryT1101.GetFieldData("t1101OutBlock", "bidho2", 0) # ---> 매도호가2
    offerho2 = instXAQueryT1101.GetFieldData("t1101OutBlock", "offerho2", 0) # ---> 매수호가2
    print("종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s" % (hname, price, bidho2, offerho2))
    print("# Price_Check End : %s, hcode : %s" % (datetime.datetime.today().strftime("%H:%M:%S"), Target_Expcode))
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : ___Price_Check ---> 종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), hname, price, bidho2, offerho2))
    output.close() # ---> close 가 되어야 write 처리가 완료된다

    instXAQueryT1101.close()

    return price, bidho2, diff
    
def __Stock_Buy_Order(Target_Expcode, Order_type, Price) :

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Order_History"+_Name+".txt"
    print("# __Stock_Buy_Order Start : %s, hcode : %s, Price : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : __Stock_Buy_Order Start ---> hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    output.close()
    ##################################################
    # 현물 정상주문
    class XAQueryEventHandlerCSPAT00600:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerCSPAT00600.query_state = 1
            
        def OnReceiveMessage(self, error, code, message):
                print("OnreceiveMessage", error, code, message)
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s : 에러 ---> %s, %s, %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), error, code, message))
                output.close() # ---> close 가 되어야 write 처리가 완료된다

    # 장기보유주식이냐 아니냐로 Buy_Qty값을 정한다
    # if Target_Expcode == "005930" or Target_Expcode == "036570" or Target_Expcode == "088980" :
    if Target_Expcode == "005935" :
        # ---> 삼성전자, 엔씨소프트, 맥쿼리인프라 ---> 일단 삼성전자(우)만 (2020.10.03)
        print("# 장기보유주식 매수신호 20만원에서 나누기")
        df_OneTime_Price = 200000
        print("df_OneTime_Price : %s, %s" %(df_OneTime_Price, type(df_OneTime_Price)))
        print("Price : %s, %s" %(Price, type(Price)))
        Buy_Qty = df_OneTime_Price/int(Price)
        print("Buy_Qty : %s" % Buy_Qty)
        Buy_Qty = int(float(Buy_Qty))# <class 'float'> ---> <class 'int'>
        print("Buy_Qty : %s" % Buy_Qty)
        print("주문수량 : %s, type : %s  : " % (Buy_Qty, type(Buy_Qty)))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 장기보유주식  매수시도 ---> 매수수량 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Buy_Qty, Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        if Buy_Qty < 1 :
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : 수량이 1이 안됨\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close() # ---> close 가 되어야 write 처리가 완료된다
            return None
    elif Target_Expcode == Preferred_Expcode_List_1[1] :
        # ---> 우선주 전략 : 현대차2우B
        print("# 우선주 매수신호 50만원에서 나누기")
        df_OneTime_Price = 500000
        print("df_OneTime_Price : %s, %s" %(df_OneTime_Price, type(df_OneTime_Price)))
        print("Price : %s, %s" %(Price, type(Price)))
        Buy_Qty = df_OneTime_Price/int(Price)
        print("Buy_Qty : %s" % Buy_Qty)
        Buy_Qty = int(float(Buy_Qty))# <class 'float'> ---> <class 'int'>
        print("Buy_Qty : %s" % Buy_Qty)
        print("주문수량 : %s, type : %s  : " % (Buy_Qty, type(Buy_Qty)))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 우선주 매수시도 ---> 매수수량 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Buy_Qty, Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        if Buy_Qty < 1 :
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : 수량이 1이 안됨\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close() # ---> close 가 되어야 write 처리가 완료된다
            return None
    else : 
        df_OneTime_Price = float(df.loc[df['종목코드'].str.match(Target_Expcode),'1회매수액'].to_string(index=False))
        Buy_Qty = df_OneTime_Price/int(Price) # ValueError: invalid literal for int() with base 10: ''
        Buy_Qty = int(float(Buy_Qty))# <class 'float'> ---> <class 'int'>

    # 호출받은 Price와 Buy_Qty의 값이 주문가능금액을 넘는지 조회
    Ready_Asset = __Ready_Asset()
    
    Call_Asset = int(Buy_Qty) * int(Price)
    if Call_Asset > int(Ready_Asset) :
        print("# %s : 주문금액 %s, 주문가능금액 %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Asset, Ready_Asset))
        print("# %s : 투자금 부족\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        _Name = datetime.datetime.today().strftime("_%Y%m%d")
        File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 투자금 부족 ---> 주문금액 %s, 주문가능금액 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Asset, Ready_Asset))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        
#         File_Name_1 = "D:\\Python\\Log\\Order_History"+_Name+".txt"
#         output.write("# %s : 투자금 부족 ---> 주문금액 %s, 주문가능금액 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Asset, Ready_Asset))
        print("# __Stock_Buy_Order End : %s, hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
#         output = open(File_Name_1, "a") # ValueError: I/O operation on closed file. ---> 바로 위 File_Name과 쫑나나?
#         output.write("# %s : __Stock_Buy_Order End   ---> hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
#         output.close()
        
        return None
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Order_History"+_Name+".txt"
    
    instXAQueryCSPAT00600 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerCSPAT00600)
    instXAQueryCSPAT00600.ResFileName = "C:\\eBEST\\xingAPI\\Res\\CSPAT00600.res"
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "AcntNo", 0, Account_No)
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "InptPwd", 0, Account_PWD)
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "IsuNo", 0, Target_Expcode)
    
    if Order_type == 2 :
        print("매수")
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdQty", 0, Buy_Qty) # 매수 수량
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdPrc", 0, Price) # 매수 가격
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "BnsTpCode", 0, 2) # 1 : 매도, 2 : 매수
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
#         output.write("# %s : 매수시도 ---> 매수종목 : %s, 매수수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), df.loc[df['종목코드'].str.match(Target_Expcode),'종목약칭'].to_string(index=False), Buy_Qty, Price, df_OneTime_Price))
        output.write("# %s : 매수시도 ---> 매수종목 : %s, 매수수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Buy_Qty, Price, df_OneTime_Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
    else :
        print("---> 매수도 매도도 아닌 상황???")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
#         output.write("# %s : 매수, 매도 ??? ---> 매수종목 : %s, 매수/매도수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), df.loc[df['종목코드'].str.match(Target_Expcode),'종목약칭'].to_string(index=False), Buy_Qty, Price, df_OneTime_Price))
        output.write("# %s : 매수, 매도 ??? ---> 매수종목 : %s, 매수/매도수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Buy_Qty, Price, df_OneTime_Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        return None
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdprcPtnCode", 0, "00") # 00 : 지정가, 03 : 시장가
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "MgntrnCode", 0, "000") # 신용거래코드 : 000 보통
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdCndiTpCode", 0, "00") # 주문조건구분 : 0 그냥 0

    instXAQueryCSPAT00600.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerCSPAT00600.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

#     ReceiveMessage = instXAQueryCSPAT00600.GetFieldData(ReceiveMessage() # AttributeError: '<win32com.client.COMEventClass instance at 0x80428624>' object has no attribute 'ReceiveMessage'
#     print(ReceiveMessage)
    
    instXAQueryCSPAT00600_count1 = instXAQueryCSPAT00600.GetBlockCount("CSPAT00600InBlock1")
    instXAQueryCSPAT00600_count2 = instXAQueryCSPAT00600.GetBlockCount("CSPAT00600InBlock2")
#     print("instXAQueryCSPAT00600_count1 : %s" % (instXAQueryCSPAT00600_count1))
#     print("instXAQueryCSPAT00600_count2 : %s" % (instXAQueryCSPAT00600_count2))
    
    RecCnt = instXAQueryCSPAT00600.GetFieldData("CSPAT00600InBlock1", "RecCnt", 0)
#     print("# 레코드 카운트 RecCnt : %s" % (RecCnt))
#     hname = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "hname", 0)
#     price = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "price", 0)
#     bidho22 = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "bidho22", 0)
#     offerho2 = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "offerho2", 0)
#     print("종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s" % (hname, price, bidho2, offerho2))
#     retrun bidho2
    
    print("# __Stock_Buy_Order End : %s, hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : __Stock_Buy_Order End   ---> hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    output.close()
    
    instXAQueryCSPAT00600.close()
    
if __name__ = '__main__' :

    Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(Preferred_Expcode_List_1[1]) # Target_Expcode의 현재 가격을 조회 (매수2호가)
    __Stock_Buy_Order(Preferred_Expcode_List_1[1], 2, Price_Check_HIDHO2) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
    


